In [1]:
# suppress warnings
%pip install stata_setup --quiet
import stata_setup
import sys
sys.path.append(r'C:/Program Files/Stata18/utilities')
from pystata import config
config.init('mp')

Note: you may need to restart the kernel to use updated packages.

  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      18.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user 4-core network, expiring 31 May 2023
Serial number: 501809302753
  Licensed to: IFPRI
               IFPRI

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [43]:
import pandas as pd
import datetime
# read stata
treatment = pd.read_stata(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\Operation\Ready for Submission\Ready for Submission\Treatment_Group_Cleaned_V0.dta')
control = pd.read_stata(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\Operation\Ready for Submission\Ready for Submission\Treatment_Group_Cleaned_V0.dta')

import os
import pandas as pd

# Set directory path
dir_path = r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\Operation\RealTimeAlerts_29 August to 26 September 2022'

# Create empty list to hold DataFrames
df_list = []

# Loop through files in directory
for file in os.listdir(dir_path):
    if file.endswith('.csv'):
        # Read CSV file into DataFrame
        df = pd.read_csv(os.path.join(dir_path, file))
        # Append DataFrame to list
        df_list.append(df)

# Concatenate all DataFrames in list into a single DataFrame
alaert_combined_df = pd.concat(df_list, ignore_index=True)

# remove column Unnamed: 0, parse Alert Time as datetime
alaert_combined_df = alaert_combined_df.drop(columns = ['Unnamed: 0'])
alaert_combined_df['Alert Time'] = pd.to_datetime(alaert_combined_df['Alert Time'])
#generate a list of row index
treatment['index']=treatment.index
#extract index,interview_time,respondent_id,c1,c1_a,c1_b
treatment=treatment[['index','interview_time','respondent_id','c1','c1_a','c1_b']]

#generate a list of row index for control
control['index']=control.index
#extract index,interview_time,respondent_id,c1,c1_a,c1_b
control=control[['index','interview_time','respondent_id','c1','c1_a','c1_b']]
#add indicator for treatment and control
treatment['treatment']=1
control['treatment']=0

#combine treatment and control
combined_df=pd.concat([treatment,control])

# read sample_interviewee.dta
sample_interviewee = pd.read_stata(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\Operation\sample_interviewees.dta')
# extract MSISDN pixel_latitude pixel_longitude 
sample_interviewee=sample_interviewee[['MSISDN','pixel_latitude','pixel_longitude']]
#rename MSISDN to respondent_id
sample_interviewee=sample_interviewee.rename(columns={'MSISDN':'respondent_id'})
# convert respondent_id to float
combined_df['respondent_id'] = combined_df['respondent_id'].astype(float)
# Convert respondent_id column to string
combined_df['respondent_id'] = combined_df['respondent_id'].astype(str)

# Add "251" to beginning of respondent_id if it is 9 digits long
combined_df['respondent_id'] = combined_df['respondent_id'].apply(lambda x: '251' + x if len(x) == 11 else x)

#cut the last 2 digits of respondent_id
combined_df['respondent_id']=combined_df['respondent_id'].str[:-2]
# convert respondent_id to float
sample_interviewee['respondent_id'] = sample_interviewee['respondent_id'].astype(float)
# Convert respondent_id column to string
sample_interviewee['respondent_id'] = sample_interviewee['respondent_id'].astype(str)

# Add "251" to beginning of respondent_id if it is 9 digits long
sample_interviewee['respondent_id'] = sample_interviewee['respondent_id'].apply(lambda x: '251' + x if len(x) == 11 else x)

#cut the last 2 digits of respondent_id
sample_interviewee['respondent_id']=sample_interviewee['respondent_id'].str[:-2]


# merge combined_df and sample_interviewee on respondent_id, keep all rows in combined_df, show _merge
combined_df = pd.merge(combined_df, sample_interviewee, on='respondent_id', how='left', indicator=True)
# value counts of _merge
combined_df['_merge'].value_counts()
#  drop left_only rows, then drop _merge column
combined_df = combined_df[combined_df['_merge'] != 'left_only']
combined_df = combined_df.drop(columns=['_merge'])
# rename pixel_latitude to latitude, pixel_longitude to longitude, rename interview_time to date
combined_df = combined_df.rename(columns={'pixel_latitude':'latitude','pixel_longitude':'longitude','interview_time':'date'})

# convert date to datetime
combined_df['date'] = pd.to_datetime(combined_df['date'])
# convert alert_time to datetime, extract date
alaert_combined_df['date'] = pd.to_datetime(alaert_combined_df['Alert Time']).dt.date


# for c1_a and c1_b that is larger than 24, divide by 100
combined_df.loc[combined_df['c1_a'] > 24, 'c1_a'] = combined_df['c1_a']/100
combined_df.loc[combined_df['c1_b'] > 24, 'c1_b'] = combined_df['c1_b']/100
combined_df.loc[combined_df['c1_b'] > 24, 'c1_b'] = combined_df['c1_b']/100

import numpy as np
# for c1_a and c1_b that is smaller than 0, add 24
combined_df.loc[combined_df['c1_a'] < 0, 'c1_a'] = combined_df['c1_a']+24
combined_df.loc[combined_df['c1_b'] < 0, 'c1_b'] = combined_df['c1_b']+24

# c1_a round to the floor , c1_b round to the ceiling, convert time
combined_df['c1_a'] = combined_df['c1_a'].apply(np.floor)-3
combined_df['c1_b'] = combined_df['c1_b'].apply(np.ceil)+3

# check if c1_b is larger than 24, if so, set c1_b to 24
combined_df.loc[combined_df['c1_b'] > 24, 'c1_b'] = 24

# check if c1_a is smaller than 0, if so, set c1_a to 0
combined_df.loc[combined_df['c1_a'] < 0, 'c1_a'] = 0

# convert c1_a and c1_b to datetime
combined_df['c1_a'] = pd.to_datetime(combined_df['c1_a'], unit='h').dt.time

combined_df['c1_b'] = pd.to_datetime(combined_df['c1_b'], unit='h').dt.time
# if c1_a or c1_b as value, set c1==1.0
combined_df.loc[combined_df['c1_a'].notnull(), 'c1'] = 1.0
combined_df.loc[combined_df['c1_b'].notnull(), 'c1'] = 1.0

#rename Longitude to longitude, Latitude to latitude
alaert_combined_df=alaert_combined_df.rename(columns={'Longitude':'longitude','Latitude':'latitude'})
# extract time from Alert Time
alaert_combined_df['time'] = pd.to_datetime(alaert_combined_df['Alert Time']).dt.time
# groupby date, latitude and longtitude, create a new column called rain, set rain to 1 if more than 50% of the rows has value more than 0 in column c1
combined_df['rain'] = combined_df.groupby(['date','latitude','longitude'])['c1'].transform(lambda x: 1 if x[x>0].count()/x.count() > 0.5 else 0)
# drop alert_time column
alaert_combined_df = alaert_combined_df.drop(columns=['Alert Time'])
#convert date to datetime
alaert_combined_df['date'] = pd.to_datetime(alaert_combined_df['date'])
combined_df['date'] = pd.to_datetime(combined_df['date'])
# merge combined_df and alaert_combined_df on date, latitude and longitude, keep all rows in combined_df, show _merge
combined_df_merge = pd.merge(combined_df, alaert_combined_df, on=['date','latitude','longitude'], how='left', indicator=True)


C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_81988\2007087936.py:123: RuntimeWarning: invalid value encountered in scalar divide
  combined_df['rain'] = combined_df.groupby(['date','latitude','longitude'])['c1'].transform(lambda x: 1 if x[x>0].count()/x.count() > 0.5 else 0)


In [44]:

# filter rain==1 and has value in column c1_a or c1_b
combined_df_merge_negative = combined_df_merge[(combined_df_merge['rain']==1) & (combined_df_merge['treatment']==1) & (combined_df_merge['c1_a'].notnull() | combined_df_merge['c1_b'].notnull())]
# Convert c1_a and c1_b columns to datetime.datetime objects
combined_df_merge_negative['c1_a'] = pd.to_datetime(combined_df_merge_negative['c1_a'], format='%H:%M:%S')
combined_df_merge_negative['c1_b'] = pd.to_datetime(combined_df_merge_negative['c1_b'], format='%H:%M:%S')

# Subtract 3 hours from c1_a and add 1 hour and 25 minutes to c1_b
combined_df_merge_negative['c1a_sc'] = combined_df_merge_negative['c1_a'] - pd.to_timedelta(3, unit='h')
combined_df_merge_negative['c1b_sc'] = combined_df_merge_negative['c1_b'] + pd.to_timedelta('1 hour 25 minutes')
# convert time to datetime
combined_df_merge_negative['time'] = pd.to_datetime(combined_df_merge_negative['time'],format='%H:%M:%S')
# Convert c1a_sc and c1b_sc columns to Timestamp objects
combined_df_merge_negative['c1a_sc'] = pd.to_datetime(combined_df_merge_negative['c1a_sc'])
combined_df_merge_negative['c1b_sc'] = pd.to_datetime(combined_df_merge_negative['c1b_sc'])

# Check if time is between c1a_sc and c1b_sc
combined_df_merge_negative['alert_yes'] = np.where((combined_df_merge_negative['time'] >= combined_df_merge_negative['c1a_sc']) & (combined_df_merge_negative['time'] <= combined_df_merge_negative['c1b_sc']), 1, 0)


C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_81988\376404894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_merge_negative['c1_a'] = pd.to_datetime(combined_df_merge_negative['c1_a'], format='%H:%M:%S')
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_81988\376404894.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_merge_negative['c1_b'] = pd.to_datetime(combined_df_merge_negative['c1_b'], format='%H:%M:%S')
C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_81988\376404894.py

In [45]:


#group by date, latitude and longitude, sum up alert_yes, set alert to 1 if alert_yes is larger than 0, else 0
combined_df_merge_negative['alert'] = combined_df_merge_negative.groupby(['date','latitude','longitude'])['alert_yes'].transform(lambda x: 1 if x.sum() > 0 else 0)
# count the percentage of alert
combined_df_merge_negative['alert'].value_counts(normalize=True)


C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_81988\841017267.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_merge_negative['alert'] = combined_df_merge_negative.groupby(['date','latitude','longitude'])['alert_yes'].transform(lambda x: 1 if x.sum() > 0 else 0)


alert
1    0.653964
0    0.346036
Name: proportion, dtype: float64

In [46]:

# summarize alert
combined_df_merge_negative.groupby(['alert'])['c1'].describe()

,count,mean,std,min,25%,50%,75%,max
alert,,,,,,,,
0,563.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,1064.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


1.False Positive: Sent Alert, how many of them are false? 2.False Negative: Claimed rains, Does the system work?

In [7]:
combined_df_merge

,index,date,respondent_id,c1,c1_a,c1_b,treatment,latitude,longitude,rain,Predicted Coverage,time,_merge
0,0,2022-09-09,251900003692,1.0,00:00:00,23:00:00,1,11.00,37.75,0,0.5,19:15:00,both
1,1,2022-09-14,251900003692,0.0,NaT,NaT,1,11.00,37.75,0,NaN,NaN,left_only
2,2,2022-09-19,251900003692,NaN,NaT,NaT,1,11.00,37.75,0,0.3,17:00:00,both
3,3,2022-09-11,251900003692,1.0,00:00:00,00:00:00,1,11.00,37.75,1,NaN,NaN,left_only
4,4,2022-09-09,251900003692,1.0,00:00:00,19:00:00,1,11.00,37.75,0,0.5,19:15:00,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16459,6522,2022-09-09,251996870449,0.0,NaT,NaT,0,11.25,38.00,0,0.4,14:00:00,both
16460,6523,2022-09-11,251996870449,0.0,NaT,NaT,0,11.25,38.00,0,NaN,NaN,left_only
16461,6524,2022-09-09,251996870449,0.0,NaT,NaT,0,11.25,38.00,0,0.4,14:00:00,both
16462,6525,2022-09-13,251996870449,1.0,00:00:00,19:00:00,0,11.25,38.00,1,0.7,14:15:00,both


In [32]:
# filter rain==0 and has value in column time
combined_df_merge_positive = combined_df_merge[(combined_df_merge['rain']==0) & (combined_df_merge['time'].notnull())& (combined_df_merge['treatment']==1)]

# filter c1==0 or c1 is null
combined_df_merge_positive = combined_df_merge_positive[(combined_df_merge_positive['c1']==0) | (combined_df_merge_positive['c1'].isnull())]


In [33]:
combined_df_merge_positive

,index,date,respondent_id,c1,c1_a,c1_b,treatment,latitude,longitude,rain,Predicted Coverage,time,_merge
2,2,2022-09-19,251900003692,NaN,NaT,NaT,1,11.00,37.75,0,0.3,17:00:00,both
5,5,2022-09-12,251900003692,0.0,NaT,NaT,1,11.00,37.75,0,0.2,13:00:00,both
6,6,2022-09-17,251900003692,0.0,NaT,NaT,1,11.00,37.75,0,0.3,20:30:00,both
13,13,2022-09-09,251900411275,NaN,NaT,NaT,1,10.50,38.00,0,0.5,15:00:00,both
23,21,2022-09-17,251900411275,NaN,NaT,NaT,1,10.50,38.00,0,0.3,14:00:00,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8211,6506,2022-09-17,251996812472,0.0,NaT,NaT,1,11.00,37.50,0,0.4,12:45:00,both
8213,6508,2022-09-16,251996812472,0.0,NaT,NaT,1,11.00,37.50,0,0.6,14:45:00,both
8226,6521,2022-09-10,251996870449,0.0,NaT,NaT,1,11.25,38.00,0,0.5,19:15:00,both
8227,6522,2022-09-09,251996870449,0.0,NaT,NaT,1,11.25,38.00,0,0.4,14:00:00,both


In [34]:
(6778-1833)/6778

0.7295662437297138

In [5]:
# unique on date, latitude and longitude, count the rows
combined_df_merge_positive.groupby(['date','latitude','longitude']).size().reset_index(name='count')

,date,latitude,longitude,count
0,2022-09-09,10.25,37.75,20
1,2022-09-09,10.50,37.50,24
2,2022-09-09,10.50,37.75,12
3,2022-09-09,10.50,38.00,18
4,2022-09-09,10.50,38.25,10
...,...,...,...,...
243,2022-09-20,12.00,38.00,2
244,2022-09-20,12.25,37.75,6
245,2022-09-20,12.50,37.00,4
246,2022-09-20,12.50,37.25,8


In [120]:
# for combined_df_merge, filter _merge==both, unique on date, latitude and longitude, count the rows

combined_df_merge_new = combined_df_merge[combined_df_merge['_merge']=='both']
combined_df_merge_new.groupby(['date','latitude','longitude']).size().reset_index(name='count')

,date,latitude,longitude,count
0,2022-09-09,10.25,37.25,32
1,2022-09-09,10.25,37.75,28
2,2022-09-09,10.50,37.50,36
3,2022-09-09,10.50,37.75,12
4,2022-09-09,10.50,38.00,18
...,...,...,...,...
597,2022-09-20,12.25,37.75,6
598,2022-09-20,12.25,38.00,4
599,2022-09-20,12.50,37.00,4
600,2022-09-20,12.50,37.25,8
